# Q3

In [ ]:
import random

DATA_WIDTH=28
DATA_HEIGHT=28
NUMBER_OF_TRAINING_EXAMPLES=5000
NUMBER_OF_VALIDATION_EXAMPLES=1000

ALL_TRAINING_IMAGES=[]
ALL_TRAINING_LABELS=[]
ALL_VALIDATION_IMAGES=[]
ALL_VALIDATION_LABELS=[]

'''
Convert ASC-II pixel into numerical data and vice versa
    - ' ' is converted to ..., which means it's part of the background
    - '#' is converted to ..., part of the image interior
    - '+' is converted to ..., part of the edges
    
'''

def _pixel_to_value(character):
    if(character == ' '):
        return 0
    elif(character == '#'):
        return 1
    elif(character == '+'):
        return 2  
    
def _value_to_pixel(value):
    if(value == 0):
        return ' '
    elif(value == 1):
        return '#'
    elif(value == 2):
        return '+'
'''
Function for loading data and label files
'''

def _load_data_file(filename, n, width, height):
    fin = [l[:-1] for l in open(filename).readlines()]
    fin.reverse()
    items = []
    for i in range(n):
        data = []
        for j in range(height):
            row = list(map(_pixel_to_value, list(fin.pop())))
            data.append(row)
        items.append(data)
    return items
        
def _load_label_file(filename, n):
    fin = [l[:-1] for l in open(filename).readlines()]
    labels = []
    for i in range(n):
        labels.append(int(fin[i]))
    return labels

def _load_all_data():

    ALL_TRAINING_IMAGES = _load_data_file("trainingimages",
        NUMBER_OF_TRAINING_EXAMPLES, DATA_WIDTH, DATA_HEIGHT)
    ALL_TRAINING_LABELS = _load_label_file("traininglabels",
        NUMBER_OF_TRAINING_EXAMPLES)

    ALL_VALIDATION_IMAGES = _load_data_file("validationimages",
        NUMBER_OF_VALIDATION_EXAMPLES, DATA_WIDTH, DATA_HEIGHT)
    ALL_VALIDATION_LABELS = _load_label_file("validationlabels",
        NUMBER_OF_VALIDATION_EXAMPLES)
    return ALL_TRAINING_IMAGES,ALL_TRAINING_LABELS,ALL_VALIDATION_IMAGES,ALL_VALIDATION_LABELS



def _print_digit_image(data):
    for row in range(len(data)):
        print(''.join(map(_value_to_pixel, data[row])))

In [ ]:
import numpy as np
import pandas as pd
import math

**1)**

In [ ]:
[x_train,y_train,x_test,y_test]=_load_all_data()


we define a function called "height_width". It calculate the number of "#"(value 1) in each row. then removes the rows that do not have any sharp signs and counts the number of remaining rows. for the width of the image, we first transopse the original piture and then execute the same algorithm

In [ ]:
def height_width(one_data):
    num_sharp_each_row=[i.count(1) for i in one_data]
    num_sharp_row_not_zero = [ele for ele in num_sharp_each_row if ele != 0]
    height=len(num_sharp_row_not_zero)

    one_data_prime=np.array(one_data).T.tolist()
    num_sharp_each_column=[i.count(1) for i in one_data_prime]
    num_sharp_column_not_zero = [ele for ele in num_sharp_each_column if ele != 0]
    width=len(num_sharp_column_not_zero)

    return height, width



we create a function that counts the number of sharp signs in the top  and bottom half ot the picture

In [ ]:
def num_sharp_top_bottom(one_data):
    top_half=one_data[0:14]
    bottom_half=one_data[14:28]
    count_top_half=sum([i.count(1) for i in top_half])
    count_bottom_half=sum([i.count(1) for i in bottom_half])
    return count_top_half, count_bottom_half

we define a function called "feature_extraction". that extracts features from one image. the features are height, width, number of # in upper half, anf number of # in bottom half 

In [ ]:
def feature_extraction(adata):
    height=[]
    width=[]
    sharp_top=[]
    sharp_bottom=[]
    for i in range(len(adata)):
        [h,w]=height_width(adata[i])
        [t,b]=num_sharp_top_bottom(adata[i])
        height.append(h)
        width.append(w)
        sharp_top.append(t)
        sharp_bottom.append(b)    
    return height, width, sharp_top, sharp_bottom

we extract the features from test and train data and use them as the new datasets

In [ ]:
[height, width, sharp_top, sharp_bottom]=feature_extraction(x_test)
df_test=pd.DataFrame({"height":height,"width":width, "top_half_sharp":sharp_top,"bottom_half_sharp":sharp_bottom,"class":y_test})


[height, width, sharp_top, sharp_bottom]=feature_extraction(x_train)
df_train=pd.DataFrame({"height":height,"width":width, "top_half_sharp":sharp_top,"bottom_half_sharp":sharp_bottom,"class":y_train})
df_train.head()

height  width  top_half_sharp  bottom_half_sharp  class
0      20     18              37                 40      5
1      20     16              49                 46      0
2      19     17              18                 28      4
3      20     12              24                 31      1
4      20     13              30                 37      9

**Naive Bayes**

Naive Bayes methods are a set of supervised learning algorithms based on applying Bayes’ theorem with the “naive” assumption of conditional independence between every pair of features given the value of the class variable. Bayes’ theorem states the following relationship, given class variable $y$ and dependent feature vectors $x1,...,x_n$:

$P(y|x_1,...,x_n)=\frac{P(y)P(x1,...,xn|y)}{P(x_1,...,x_n)}$

Using the naive conditional independence assumption that:

$P(x_i|y,x_1,...,x_{i-1},x_{i+1},...,x_n)=P(x_i|y)$ $\rightarrow$ 

  $P(y|x_1,...,x_n)=\frac{P(y) \prod_{i=1}^{n}P(x_i|y)}{P(x_1,...,x_n)}$

for classification:
 $\hat{y}=argmax_y\  \ P(y|x_1,...,x_n)=argmax_y\  \ \frac{P(y) \prod_{i=1}^{n}P(x_i|y)}{P(x_1,...,x_n)}$

if we have $c$ claase:

 $P(x_1,...,x_n)$ is constant for $y_1,..,y_c$ we also have no prior information, so $P(y_1)=...=P(y_c)=\frac{1}{c}$ so $P(y)$ is also the same for all $y_1,...,y_n$

so we can say:

 $\hat{y}=argmax_y\  \  \prod_{i=1}^{n}P(x_i|y)$


 naive bayesian method is a parametric estimation that assumes likelihood ($P(x_i|y_j)$) has a normal distribution. we use the MLE method to estimate parameters of the normal distribution for each feature vector $x_i$ and class label $y_j$

MLE:

consider $D=\{ \omega_1,..., \omega_m \}$ as all the datapoints of feature vector $x_i$ that are labeld as $y_j$, we know that they have a normal distribution, we estimate the parameters of the distribution ( $ \theta= [ \mu , \sigma]^T$) as below:

$L(\theta | D)=P(D|\theta)=P(\omega_1,...,\omega_m|\theta)=\prod_{i=1}^{m}P(\omega_i|\theta)$

$\hat{\theta}=argmax\ L(D|\theta)=argmax\ log L(D|\theta)=argmax\ \Sigma_{i=1}^{m}log\ P(w_i|\theta)$   

$P(\omega_i|\theta)=\frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{1}{2}(\frac{\omega_i-\mu}{\sigma})^2}$ $\rightarrow$ $log P(\omega_i|\theta)= C-log(\sigma)-\frac{1}{2}(\frac{\omega_i-\mu}{\sigma})^2$   

$\nabla \Sigma_{i=1}^{m}log\ P(w_i|\theta)=\nabla \Sigma_{i=1}^{m}C-log(\sigma)-\frac{1}{2}(\frac{\omega_i-\mu}{\sigma})^2=0$ 

$\implies \hat{\mu}=\frac{1}{m}\Sigma_{i=1}^{m}\omega_i$,

$\implies  \hat{\sigma}^2=\frac{1}{m}\Sigma_{i=1}^{m}(\omega_i-\hat{\mu})^2$  


we define a function called "calculate_probability_single_feature" that for a specific feature f and a specific class c, gets the mean and std of the corresponding datapoints to estimate the probability using the guassian distribution.

In [ ]:
def calculate_probability_single_feature(single_feature, mean, std):
    exponent = math.exp((-1/2) * (((single_feature - mean) / std) ** 2))
    GaussianProbability = (1 / ((math.sqrt(2 * math.pi)) * std)) * exponent
    return GaussianProbability

since features are assumed to be independent from each other in naive bayesian methode, we define a function called "calculate_probability_all_feature_one_class"  that for each datapoint, for a specific class c, calculates the probability of all features , multiplies all of the probabilities and rturn it. the returned result is the probability that th datapoint belongs to that specific class

In [ ]:
def calculate_probability_all_feature_one_class(single_data,mean_one_class,std_one_class,feature_names):
    p_all=1
    for name in feature_names:
        p_feature=calculate_probability_single_feature(single_data[name], mean_one_class[name], std_one_class[name])
        p_all=p_all*p_feature
    
    return p_all

we define a function called "classify_one_data" that calculate the probability that a single data point belongs to each class and returns the class name that has the most probability.

In [ ]:
def classify_one_data(single_data,mean_all_class,std_all_class,feature_names,class_labels):
    p_all_class=[]
    for i in range(len(class_labels)):
        mean_one_class=mean_all_class[i]
        std_one_class=std_all_class[i]
        p_one_class=calculate_probability_all_feature_one_class(single_data,mean_one_class,std_one_class,feature_names)
        p_all_class.append(p_one_class)
    p_all_class=np.array(p_all_class)
    y_est=np.argmax(p_all_class)
    return y_est



we define a function called "classify_all_data" that classifies all datapoints

In [ ]:
def classify_all_data(all_data,mean_all_class,std_all_class,feature_names,class_labels):
    y_est_all=[]
    for i in range(len(all_data)):
        single_data=all_data.iloc[i]
        y_est=classify_one_data(single_data,mean_all_class,std_all_class,feature_names,class_labels)
        y_est_all.append(y_est)
    return y_est_all

we define a function called "calculate_accuracy" that calculates the accuray of the estimated labels

In [ ]:
def calculate_accuracy(y_est,y):
    count=0
    n=len(y)
    for i in range(n):
        if y[i]==y_est[i]:
            count=count +1
    accur=(count/n)*100
    return accur


first we find all the classes

In [ ]:
class_labels =df_train["class"].unique()
class_labels.sort()
print("classes")
print(class_labels)

classes
[0 1 2 3 4 5 6 7 8 9]


then we seperate our dataset based on their class label.

In [ ]:
x_train_all_classes=[]
for i in range(len(class_labels)):
    x_train_class_i=df_train.loc[df_train["class"] == class_labels[i]]
    x_train_class_i.reset_index()
    x_train_all_classes.append(x_train_class_i)

we assume all distributions are normal so we calculate the mean and variance of the training datapoints in each class. (for class and for each feature, we calculate the mean and std) as the parameters of the distributions

In [ ]:
means_all_class=[]
stds_all_class=[]
for i in range(len(class_labels)):
    means=x_train_all_classes[i].mean(axis=0)
    stds=x_train_all_classes[i].std(axis=0)
    means_all_class.append(means)
    stds_all_class.append(stds)

now we classify the datapoints using the estimated models

In [ ]:
feature_names=["height", "width", "top_half_sharp", "bottom_half_sharp"]

In [ ]:
y_est_train=classify_all_data(df_train,means_all_class,stds_all_class,feature_names,class_labels)
y_est_test=classify_all_data(df_test,means_all_class,stds_all_class,feature_names,class_labels)

**2)**

In [ ]:
train_accur=calculate_accuracy(y_est_train,y_train)
test_accur=calculate_accuracy(y_est_test,y_test)
print("training accuracy: "+str(train_accur))
print("test accuracy: "+str(test_accur))

training accuracy: 35.08
test accuracy: 29.099999999999998


we can see that training accuracy is 35.08% and the test accuracy is 29.09%. which is very low.
we assumed that all classes have normal density distribution. but the distributions could be more complicated. we also assumed that our featurse are independent which could be not true. as a result, both test and train accuracies are low

also the extracted features might not be relevant for this classification

refrence:


H. Zhang (2004). The optimality of Naive Bayes. Proc. FLAIRS.